In [1]:
import pandas as pd

In [40]:
cleaned_data = pd.read_csv('diabetes_data.csv')
dia_data = cleaned_data[(cleaned_data['QuestionID'] == 'DIA01') & 
                        (cleaned_data['StratificationCategoryID1'] == 'OVERALL') &
                        (cleaned_data['LocationAbbr'] == 'AL') &
                        (cleaned_data['Year'] == 2021)]
dia_data

,Unnamed: 0.1,Unnamed: 0,LocationAbbr,DataSource,DataValueUnit,DataValue,DataValueAlt,LowConfidenceLimit,HighConfidenceLimit,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,Year
6640,6640,177592,AL,BRFSS,%,13.5,13.5,12.3,14.8,DIA,DIA01,AGEADJPREV,OVERALL,OVR,2021
6786,6786,180705,AL,BRFSS,%,15.1,15.1,13.8,16.4,DIA,DIA01,CRDPREV,OVERALL,OVR,2021


In [46]:
npw_data = cleaned_data[(cleaned_data['QuestionID'] == 'DIA01') & 
                        (cleaned_data['StratificationCategoryID1'] == 'OVERALL') &
                        (cleaned_data['DataValueTypeID'] == 'CRDPREV') &
                        # (cleaned_data['LocationAbbr'] == 'AL') &
                        (cleaned_data['Year'] == 2021)]
npw_data[['LocationAbbr', 'DataValue']]

,LocationAbbr,DataValue
6467,AK,8.3
6690,CA,11.6
6736,AZ,11.0
6752,AR,12.5
6786,AL,15.1
7025,HI,9.5
7049,GA,12.3
7089,CT,10.8
7196,CO,7.0
7212,DC,7.7


In [48]:
import plotly.graph_objs as go
import plotly.io as pio

# Create a bar plot
fig = go.Figure(data=[go.Bar(
    x=npw_data['LocationAbbr'],  # LocationAbbr on x-axis
    y=npw_data['DataValue'],     # DataValue on y-axis
    marker_color='skyblue'       # Set color of the bars
)])

# Update layout
fig.update_layout(
    title='Diabetes among adults',
    xaxis_title='Location',
    yaxis_title='Percentage',
    xaxis=dict(tickangle=45)     # Rotate x-axis labels for better readability
)

# Show the plot
pio.show(fig)